In [8]:
import cv2
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model

In [9]:
def localization_loss(y_true, y_pred):
    # Coordinate differences
    delta_coord = tf.reduce_sum(tf.square(y_true[:, :2] - y_pred[:, :2]))
    
    # Height and width differences
    h_true = y_true[:, 3] - y_true[:, 1]
    w_true = y_true[:, 2] - y_true[:, 0]

    h_pred = y_pred[:, 3] - y_pred[:, 1]
    w_pred = y_pred[:, 2] - y_pred[:, 0]

    delta_size = tf.reduce_sum(tf.square(w_true - w_pred) + tf.square(h_true - h_pred))

    return delta_coord + delta_size

In [10]:
facetracker = load_model('facetracker.h5', custom_objects={'localization_loss': localization_loss})

In [ ]:
cap=cv2.VideoCapture(0)
while cap.isOpened():
    _,frame=cap.read()
    frame=frame[50:500,50:500,:]#we need 450*450px
    
    rgb=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    resized=tf.image.resize(rgb,(120,120))
    
    y_pred=facetracker.predict(np.expand_dims(resized/255,0))
    sample_coords=y_pred[1][0]
    
    if y_pred[0]>0.5:
        
        cv2.rectangle(frame,
            tuple(np.multiply(sample_coords[:2],[450,450]).astype(int)),
            tuple(np.multiply(sample_coords[2:], [450, 450]).astype(int)),
            (255,0,0), 2
        )
        cv2.rectangle(frame, 
            tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int), 
                [0,-30])),
            tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
                [80,0])), 
            (255,0,0), -1
        )
        
        cv2.putText(frame, 'face', tuple(np.add(np.multiply(sample_coords[:2], [450,450]).astype(int),
            [0,-5])),
            cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA
        )
        
    cv2.imshow('EyeTrack',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()